In [1]:
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.0/165.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 15.0.1
    Uninstalling websockets-15.0.1:
      Successfully uninstalled websockets-15.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.17.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 13.1 which is incompatible.
dataproc-spark-connect 0.8.3 requires websockets>=14.0, but you have websockets 13.1 which is incompatible.


In [2]:
import fastf1
import pandas as pd
import matplotlib.pyplot as plt
import os

# Enable Cache
if not os.path.exists('cache'):
  os.makedirs('cache')
fastf1.Cache.enable_cache('cache')
session = fastf1.get_session(2025, 'United States Grand Prix', 'R')
session.load()

# Pick both drivers' clean laps only
lando = session.laps.pick_driver('NOR').pick_quicklaps()
charles = session.laps.pick_driver('LEC').pick_quicklaps()

# Convert lap times to seconds
lando.loc[:, 'LapTime_s'] = lando['LapTime'].dt.total_seconds()
charles.loc[:, 'LapTime_s'] = charles['LapTime'].dt.total_seconds()

# Focus only on laps 10 to 21
lando_window = lando[(lando['LapNumber'] >= 10) & (lando['LapNumber'] <= 21)]
charles_window = charles[(charles['LapNumber'] >= 10) & (charles['LapNumber'] <= 21)]

# Merge by lap number
merged = pd.merge(
    lando_window[['LapNumber', 'LapTime_s']],
    charles_window[['LapNumber', 'LapTime_s']],
    on='LapNumber',
    suffixes=('_NOR', '_LEC')
)

# Calculate per-lap and cumulative delta
merged['Delta_per_lap'] = merged['LapTime_s_LEC'] - merged['LapTime_s_NOR']
merged['Cumulative_gain'] = merged['Delta_per_lap'].cumsum()

# Display table of lap-by-lap gain
print(merged[['LapNumber', 'LapTime_s_NOR', 'LapTime_s_LEC', 'Delta_per_lap', 'Cumulative_gain']])

# Total gain between Lap 10 and Lap 21
total_gain = merged['Cumulative_gain'].iloc[-1]
print(f"\n➡️ Between Lap 10 and 21, Norris gained {total_gain:.2f} seconds on Leclerc.")


events      WARNING 	Correcting user input 'United States Grand Prix' to 'United States Grand Prix'
core           INFO 	Loading data for United States Grand Prix - Race [v3.6.1]
INFO:fastf1.fastf1.core:Loading data for United States Grand Prix - Race [v3.6.1]
req            INFO 	No cached data found for session_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
INFO:fastf1.api:Fetching session info data...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
INFO:fastf1.fastf1.req:No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
INFO:fastf1.api:Fetching driver list...
req            INFO 	Data has been written to cache!
INFO:fastf1.fastf1.req:Data has been written to cache!
req            INFO 	No cached data

    LapNumber  LapTime_s_NOR  LapTime_s_LEC  Delta_per_lap  Cumulative_gain
0        10.0         99.333         99.378          0.045            0.045
1        11.0         99.695        100.029          0.334            0.379
2        12.0         99.775        100.173          0.398            0.777
3        13.0         99.712         99.797          0.085            0.862
4        14.0         99.885        100.231          0.346            1.208
5        15.0        101.266        101.331          0.065            1.273
6        16.0        100.626        100.371         -0.255            1.018
7        17.0         99.905        100.177          0.272            1.290
8        18.0        100.087         99.874         -0.213            1.077
9        19.0        100.603        100.690          0.087            1.164
10       20.0        100.061        100.055         -0.006            1.158
11       21.0        100.082        101.601          1.519            2.677

➡️ Between 

/usr/local/lib/python3.12/dist-packages/fastf1/core.py:3183: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
